<a href="https://colab.research.google.com/github/Joe270585/Transfer_Learning/blob/main/Projeto_Treinamento_Redes_Neurais_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Imagens: Gatos vs Cachorros

Este projeto demonstra a construção e treinamento de um modelo de aprendizado de máquina para classificar imagens como "gato" ou "cachorro" utilizando Transfer Learning com o modelo MobileNetV2 e o dataset Cats vs Dogs do TensorFlow Datasets.

## Requisitos

Para executar este código, você precisará ter as seguintes bibliotecas instaladas no seu ambiente Python:

- `tensorflow`
- `tensorflow-datasets`
- `numpy`
- `matplotlib`
- `Pillow` (PIL Fork)

Você pode instalar essas dependências usando pip:

In [ ]:
# Importar as bibliotecas
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Carregar o dataset Cats vs Dogs e dividir em treino, validação e teste

(train_ds, val_ds, test_ds), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.8VLLQE_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [ ]:
# Redimensionar as imagens para o tamanho que o modelo espera e normaliza os pixels

IMG_SIZE = (160, 160)

def format_image(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(format_image).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(format_image).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(format_image).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Carregar o modelo pré-treinado MobileNetV2 com transfer learning

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Treinar o modelo por 4 épocas

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=4)

Epoch 1/4
509/509 ━━━━━━━━━━━━━━━━━━━━ 359s 694ms/step - accuracy: 0.9546 - loss: 0.1310 - val_accuracy: 0.9782 - val_loss: 0.0606
Epoch 2/4
509/509 ━━━━━━━━━━━━━━━━━━━━ 345s 678ms/step - accuracy: 0.9798 - loss: 0.0576 - val_accuracy: 0.9797 - val_loss: 0.0579
Epoch 3/4
509/509 ━━━━━━━━━━━━━━━━━━━━ 359s 706ms/step - accuracy: 0.9820 - loss: 0.0510 - val_accuracy: 0.9808 - val_loss: 0.0571
Epoch 4/4
509/509 ━━━━━━━━━━━━━━━━━━━━ 360s 707ms/step - accuracy: 0.9837 - loss: 0.0470 - val_accuracy: 0.9808 - val_loss: 0.0569


In [ ]:
# Avaliar o modelo treinado

model.evaluate(test_ds)

# Salvar o modelo treinado
model.save('cats_vs_dogs_model.h5')

# Baixar o modelo para o computador
from google.colab import files
files.download('cats_vs_dogs_model.h5')

110/110 ━━━━━━━━━━━━━━━━━━━━ 59s 529ms/step - accuracy: 0.9767 - loss: 0.0626


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Escolher a foto para teste

from google.colab import files
uploaded = files.upload()

In [ ]:
# Carregar o modelo treinado
model = tf.keras.models.load_model('cats_vs_dogs_model.h5')

# Obtendo o primeiro nome do arquivo carregado
img_path = list(uploaded.keys())[0]
print(f"Trying to open: {img_path}")

# Preparar a imagem
img = Image.open(img_path).resize((160, 160))
img_array = np.array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Fazer a previsão
prediction = model.predict(img_array)

# Mostrar imagem e resultado
plt.imshow(img)
plt.axis('off')
plt.title("Imagem testada")
plt.show()

# Classificar o resultado
if prediction[0][0] > 0.5:
    print("CACHORRO")
else:
    print("GATO")